In [5]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>

### softmax简洁版本实现，与公开课稍微有些不同

常规的CPU实现，一个全连接层再加一个softmax完成多分类；

In [6]:
/*design a net*/
struct Net : torch::nn::Module {
  Net() {
    // Construct and register two Linear submodules.
    fc = register_module("fc", torch::nn::Linear(784, 10));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {
    // Use one of many tensor manipulation functions.
    x = torch::log_softmax(fc->forward(x.reshape({x.size(0), 784})), /*dim=*/1);
    return x;
  }

  // Use one of many "standard library" modules.
  torch::nn::Linear fc{nullptr};
};

In [8]:
// Create a new Net.
auto net = std::make_shared<Net>();

// Create a multi-threaded data loader for the MNIST dataset.
auto data_loader = torch::data::make_data_loader(
  torch::data::datasets::MNIST("../dataset/fashion_mnist").map(
      torch::data::transforms::Stack<>()),
  /*batch_size=*/100);

// Instantiate an SGD optimization algorithm to update our Net's parameters.
torch::optim::SGD optimizer(net->parameters(), /*lr=*/0.01);

for (size_t epoch = 1; epoch <= 20; ++epoch) {
    size_t batch_index = 0;
    // Iterate the data loader to yield batches from the dataset.
    for (auto& batch : *data_loader) {
        // Reset gradients.
        optimizer.zero_grad();
        // Execute the model on the input data.
        torch::Tensor prediction = net->forward(batch.data);
        // Compute a loss value to judge the prediction of our model.
        torch::Tensor loss = torch::nll_loss(prediction, batch.target);
        // Compute gradients of the loss w.r.t. the parameters of our model.
        loss.backward();
        // Update the parameters based on the calculated gradients.
        optimizer.step();
        // Output the loss and checkpoint every 100 batches.
        if (++batch_index % 500 == 0) {
        std::cout << "Epoch: " << epoch << " | Batch: " << batch_index
                  << " | Loss: " << loss.item<float>() << std::endl;
        // Serialize your model periodically as a checkpoint.
        torch::save(net, "net.pt");
        }
    }    
}

std::cout << std::endl << "\r\nTraining finished!\r\n" << std::endl;

Epoch: 1 | Batch: 500 | Loss: 0.824684
Epoch: 2 | Batch: 500 | Loss: 0.722425
Epoch: 3 | Batch: 500 | Loss: 0.731193
Epoch: 4 | Batch: 500 | Loss: 0.581636
Epoch: 5 | Batch: 500 | Loss: 0.661941
Epoch: 6 | Batch: 500 | Loss: 0.4241
Epoch: 7 | Batch: 500 | Loss: 0.505224
Epoch: 8 | Batch: 500 | Loss: 0.462436
Epoch: 9 | Batch: 500 | Loss: 0.439719
Epoch: 10 | Batch: 500 | Loss: 0.466451
Epoch: 11 | Batch: 500 | Loss: 0.523327
Epoch: 12 | Batch: 500 | Loss: 0.403232
Epoch: 13 | Batch: 500 | Loss: 0.506222
Epoch: 14 | Batch: 500 | Loss: 0.441393
Epoch: 15 | Batch: 500 | Loss: 0.519449
Epoch: 16 | Batch: 500 | Loss: 0.488209
Epoch: 17 | Batch: 500 | Loss: 0.513735
Epoch: 18 | Batch: 500 | Loss: 0.396346
Epoch: 19 | Batch: 500 | Loss: 0.516452
Epoch: 20 | Batch: 500 | Loss: 0.724814


Training finished!



备注：后续加上图形化训练结果以及处理时间